In [ ]:
'''/content/drive/MyDrive/Colab Notebooks/PROJECT/dataset/train   FOR MY RGB IMAGES
/content/drive/MyDrive/Colab Notebooks/PROJECT/hyperspectral_dataset/train  FOR MY HSI IMAGES
/content/drive/MyDrive/Colab Notebooks/PROJECT/models/hyperspectral_model_finetuned.pth   THIS MY WEIGHTES FILE
/content/drive/MyDrive/Colab Notebooks/PROJECT/models/MST_Plus_Plus.py    THIS IS MY MSTPP.PY FILE
/content/drive/MyDrive/Colab Notebooks/PROJECT/paired_files_by_class.txt   PAIRED FILE PATH '''

In [ ]:
import os
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)
print("DEBUG: Google Drive mounted", flush=True)

# Paths
rgb_base = "/content/drive/MyDrive/Colab Notebooks/PROJECT/dataset/train"
hsi_base = "/content/drive/MyDrive/Colab Notebooks/PROJECT/hyperspectral_dataset/train"
output_file = "/content/drive/MyDrive/Colab Notebooks/PROJECT/paired_files_by_class_new.txt"

# Class name mapping (expected name -> folder name)
class_mapping = {
    'actinic keratosis': 'actinic_keratosis',
    'basal cell carcinoma': 'bcc',
    'dermatofibroma': 'dermatofibroma',
    'melanoma': 'melanoma',
    'nevus': 'nevus',
    'pigmented benign keratosis': 'pigmented_benign_keratosis',
    'seborrheic keratosis': 'seborrheic_keratosis',
    'squamous cell carcinoma': 'scc',
    'vascular lesion': 'vascular_lesion'
}

# Collect valid pairs
valid_pairs = []
for expected_name, folder_name in class_mapping.items():
    rgb_dir = os.path.join(rgb_base, folder_name)
    hsi_dir = os.path.join(hsi_base, folder_name)

    if not os.path.exists(rgb_dir) or not os.path.exists(hsi_dir):
        print(f"DEBUG: Skipping class {expected_name}: RGB or HSI directory missing")
        continue

    # List RGB files
    rgb_files = [f for f in os.listdir(rgb_dir) if f.startswith('ISIC_') and f.lower().endswith(('.jpg', '.png', '.jpeg'))]
    rgb_ids = [f.split('.')[0] for f in rgb_files]  # e.g., ISIC_0025780

    # List HSI files
    hsi_files = [f for f in os.listdir(hsi_dir) if f.startswith('ISIC_') and f.endswith('_hs.npy')]
    hsi_ids = [f.replace('_hs.npy', '') for f in hsi_files]  # e.g., ISIC_0025780

    # Find matching pairs
    common_ids = sorted(set(rgb_ids) & set(hsi_ids))
    if not common_ids:
        print(f"DEBUG: No matching pairs for class {expected_name}")
        continue

    print(f"DEBUG: Found {len(common_ids)} pairs for class {expected_name}")
    valid_pairs.append((expected_name, common_ids))

# Write new paired file
with open(output_file, 'w') as f:
    for class_name, ids in valid_pairs:
        f.write(f"Class: {class_name} - {len(ids)} images\n")
        for id in ids:
            f.write(f"{id}\n")
print(f"DEBUG: New paired file saved to {output_file}")

# Count total pairs
total_pairs = sum(len(ids) for _, ids in valid_pairs)
print(f"DEBUG: Total valid pairs: {total_pairs}")

Mounted at /content/drive
DEBUG: Google Drive mounted
DEBUG: Found 114 pairs for class actinic keratosis
DEBUG: Found 376 pairs for class basal cell carcinoma
DEBUG: Found 95 pairs for class dermatofibroma
DEBUG: Found 438 pairs for class melanoma
DEBUG: Found 357 pairs for class nevus
DEBUG: Found 462 pairs for class pigmented benign keratosis
DEBUG: Found 77 pairs for class seborrheic keratosis
DEBUG: Found 181 pairs for class squamous cell carcinoma
DEBUG: Found 139 pairs for class vascular lesion
DEBUG: New paired file saved to /content/drive/MyDrive/Colab Notebooks/PROJECT/paired_files_by_class_new.txt
DEBUG: Total valid pairs: 2239


In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.amp import GradScaler, autocast
from PIL import Image
import numpy as np
import torchvision.transforms as transforms
import logging
import importlib.util
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)
print("DEBUG: Google Drive mounted", flush=True)

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)
print("DEBUG: Starting fine-tuning script...", flush=True)

# File paths
weights_file = "/content/drive/MyDrive/Colab Notebooks/PROJECT/models/hyperspectral_model_finetuned.pth"
mstpp_py_file = "/content/drive/MyDrive/Colab Notebooks/PROJECT/models/MST_Plus_Plus.py"
paired_file = "/content/drive/MyDrive/Colab Notebooks/PROJECT/paired_files_by_class_new.txt"
rgb_base = "/content/drive/MyDrive/Colab Notebooks/PROJECT/dataset/train"
hsi_base = "/content/drive/MyDrive/Colab Notebooks/PROJECT/hyperspectral_dataset/train"

# Verify file existence
for path in [weights_file, mstpp_py_file, paired_file]:
    if not os.path.exists(path):
        raise FileNotFoundError(f"File not found: {path}")
print("DEBUG: All files found", flush=True)

# Import MST_Plus_Plus model
spec = importlib.util.spec_from_file_location("mstpp", mstpp_py_file)
module = importlib.util.module_from_spec(spec)
spec.loader.exec_module(module)
MST_Plus_Plus = getattr(module, "MST_Plus_Plus")

# Custom classification wrapper
class MSTPPClassifier(nn.Module):
    def __init__(self, base_model, num_classes=9):
        super(MSTPPClassifier, self).__init__()
        self.base = base_model
        self.pool = nn.AdaptiveAvgPool2d(1)
        # Get the output channels dynamically
        self.base.eval()
        with torch.no_grad():
            sample_input = torch.zeros(1, 3, 256, 256).cuda()  # Dummy RGB input
            sample_output = self.base(sample_input)
            in_features = sample_output.shape[1]
            print(f"DEBUG: Base model output channels: {in_features}")
        self.classifier = nn.Linear(in_features, num_classes)

    def forward(self, x):
        x = self.base(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# Dataset class
class SkinCancerDataset(Dataset):
    def __init__(self, paired_file, rgb_base, hsi_base, transform=None):
        self.pairs = []
        self.labels = []
        self.folder_mapping = {
            'actinic keratosis': 'actinic_keratosis',
            'basal cell carcinoma': 'bcc',
            'dermatofibroma': 'dermatofibroma',
            'melanoma': 'melanoma',
            'nevus': 'nevus',
            'pigmented benign keratosis': 'pigmented_benign_keratosis',
            'seborrheic keratosis': 'seborrheic_keratosis',
            'squamous cell carcinoma': 'scc',
            'vascular lesion': 'vascular_lesion'
        }
        self.class_to_idx = {
            'actinic keratosis': 0,
            'basal cell carcinoma': 1,
            'dermatofibroma': 2,
            'melanoma': 3,
            'nevus': 4,
            'pigmented benign keratosis': 5,
            'seborrheic keratosis': 6,
            'squamous cell carcinoma': 7,
            'vascular lesion': 8
        }
        self.transform = transform

        if not os.path.isfile(paired_file):
            raise FileNotFoundError(f"Paired file not found: {paired_file}")

        with open(paired_file, "r") as f:
            lines = f.readlines()
            current_class = None
            for line in lines:
                line = line.strip()
                if line.startswith("Class:"):
                    current_class = line.split(" - ")[0].replace("Class: ", "").strip()
                elif line.startswith("ISIC_") and current_class in self.class_to_idx:
                    self.pairs.append((current_class, line))
                    self.labels.append(self.class_to_idx[current_class])

        # Validate files
        print("DEBUG: Validating dataset files...", flush=True)
        missing_files = []
        for class_name, filename in self.pairs:
            folder_name = self.folder_mapping[class_name]
            rgb_path = None
            for ext in ['.jpg', '.png', '.jpeg', '.JPG', '.JPEG']:
                temp_path = os.path.join(rgb_base, folder_name, f"{filename}{ext}")
                if os.path.isfile(temp_path):
                    rgb_path = temp_path
                    break
            if not rgb_path:
                missing_files.append((class_name, filename, f"{rgb_base}/{folder_name}/{filename}"))
            hsi_path = os.path.join(hsi_base, folder_name, f"{filename}_hs.npy")
            if not os.path.isfile(hsi_path):
                missing_files.append((class_name, filename, hsi_path))
        if missing_files:
            for class_name, filename, path in missing_files:
                print(f"DEBUG: Missing file: {path}", flush=True)
            raise FileNotFoundError(f"{len(missing_files)} files missing, see logs above")

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        class_name, filename = self.pairs[idx]
        label = self.labels[idx]
        folder_name = self.folder_mapping[class_name]

        # Load RGB image
        rgb_path = None
        for ext in ['.jpg', '.png', '.jpeg', '.JPG', '.JPEG']:
            temp_path = os.path.join(rgb_base, folder_name, f"{filename}{ext}")
            if os.path.isfile(temp_path):
                rgb_path = temp_path
                break
        if not rgb_path:
            raise FileNotFoundError(f"RGB image not found for {filename} at {rgb_base}/{folder_name}")

        rgb_img = Image.open(rgb_path).convert('RGB')

        # Load HSI image (for validation)
        hsi_path = os.path.join(hsi_base, folder_name, f"{filename}_hs.npy")
        if not os.path.isfile(hsi_path):
            raise FileNotFoundError(f"HSI image not found: {hsi_path}")
        hsi_data = np.load(hsi_path)[:, :, :31]

        # Preprocess
        if self.transform:
            rgb_img = self.transform(rgb_img)
        hsi_data = torch.from_numpy(hsi_data).float().permute(2, 0, 1)
        hsi_data = torch.nn.functional.interpolate(hsi_data.unsqueeze(0), size=(256, 256), mode='bilinear', align_corners=False).squeeze(0)

        return rgb_img, hsi_data, label

# Setup dataset and dataloader
rgb_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

try:
    dataset = SkinCancerDataset(paired_file, rgb_base, hsi_base, transform=rgb_transforms)
    dataloader = DataLoader(dataset, batch_size=16, shuffle=True, num_workers=2, pin_memory=True)
    print("DEBUG: Dataset and dataloader created", flush=True)
except Exception as e:
    print(f"DEBUG: Failed to create dataset: {str(e)}", flush=True)
    logging.error(f"Failed to create dataset: {str(e)}")
    exit(1)

# Load model and weights
try:
    model = MSTPPClassifier(MST_Plus_Plus().cuda()).cuda() # Move the base model to cuda here
    state_dict = torch.load(weights_file, map_location='cpu', weights_only=True)
    # Move weights to GPU
    for key in state_dict:
        state_dict[key] = state_dict[key].cuda()
    model.load_state_dict(state_dict, strict=False)
    print("DEBUG: Model loaded successfully", flush=True)
    print(f"DEBUG: GPU memory allocated after model load: {torch.cuda.memory_allocated() / 1e9:.2f} GB", flush=True)
    logging.info("Model loaded successfully")
except Exception as e:
    print(f"DEBUG: Failed to load model: {str(e)}", flush=True)
    logging.error(f"Failed to load model: {str(e)}")
    # Instead of exit(1), raise the exception to stop execution
    raise e

# Training setup (inside try block to ensure model is defined)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scaler = GradScaler('cuda')
num_epochs = 10
accumulation_steps = 2

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    try:
        for i, (rgb, hsi, labels) in enumerate(dataloader):
            rgb, hsi, labels = rgb.cuda(), hsi.cuda(), labels.cuda()
            print(f"DEBUG: Batch {i+1}, RGB shape: {rgb.shape}, HSI shape: {hsi.shape}", flush=True)
            print(f"DEBUG: GPU memory allocated: {torch.cuda.memory_allocated() / 1e9:.2f} GB", flush=True)

            optimizer.zero_grad(set_to_none=True)
            with autocast('cuda'):
                outputs = model(rgb)
                print(f"DEBUG: Model output shape: {outputs.shape}")  # Debug output shape
                loss = criterion(outputs, labels) / accumulation_steps
            scaler.scale(loss).backward()

            if (i + 1) % accumulation_steps == 0:
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad(set_to_none=True)

            running_loss += loss.item() * accumulation_steps * rgb.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    except Exception as e:
        print(f"DEBUG: Error during training: {str(e)}", flush=True)
        logging.error(f"Error during training: {str(e)}")
        break

    epoch_loss = running_loss / len(dataset)
    epoch_acc = 100 * correct / total
    print(f"DEBUG: Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%", flush=True)
    logging.info(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")

# Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for rgb, hsi, labels in dataloader:
        rgb, hsi, labels = rgb.cuda(), hsi.cuda(), labels.cuda()
        outputs = model(rgb)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

eval_acc = 100 * correct / total
print(f"DEBUG: Evaluation Accuracy: {eval_acc:.2f}%")
logging.info(f"Evaluation Accuracy: {eval_acc:.2f}%")

# Save fine-tuned model
try:
    save_path = "/content/drive/MyDrive/Colab Notebooks/PROJECT/models/mstpp_finetuned_new.pth"
    torch.save(model.state_dict(), save_path)
    print(f"DEBUG: Model saved to {save_path}", flush=True)
    logging.info("Model saved successfully")
except Exception as e:
    print(f"DEBUG: Failed to save model: {str(e)}", flush=True)
    logging.error(f"Failed to save model: {str(e)}")

Mounted at /content/drive
DEBUG: Google Drive mounted
DEBUG: Starting fine-tuning script...
DEBUG: All files found
DEBUG: Validating dataset files...
DEBUG: Dataset and dataloader created
DEBUG: Base model output channels: 31
DEBUG: Model loaded successfully
DEBUG: GPU memory allocated after model load: 0.02 GB


2025-04-30 10:20:48,938 - INFO - Model loaded successfully


DEBUG: Batch 1, RGB shape: torch.Size([16, 3, 256, 256]), HSI shape: torch.Size([16, 31, 256, 256])
DEBUG: GPU memory allocated: 0.17 GB
DEBUG: Model output shape: torch.Size([16, 9])
DEBUG: Batch 2, RGB shape: torch.Size([16, 3, 256, 256]), HSI shape: torch.Size([16, 31, 256, 256])
DEBUG: GPU memory allocated: 0.32 GB
DEBUG: Model output shape: torch.Size([16, 9])
DEBUG: Batch 3, RGB shape: torch.Size([16, 3, 256, 256]), HSI shape: torch.Size([16, 31, 256, 256])
DEBUG: GPU memory allocated: 0.33 GB
DEBUG: Model output shape: torch.Size([16, 9])
DEBUG: Batch 4, RGB shape: torch.Size([16, 3, 256, 256]), HSI shape: torch.Size([16, 31, 256, 256])
DEBUG: GPU memory allocated: 0.34 GB
DEBUG: Model output shape: torch.Size([16, 9])
DEBUG: Batch 5, RGB shape: torch.Size([16, 3, 256, 256]), HSI shape: torch.Size([16, 31, 256, 256])
DEBUG: GPU memory allocated: 0.33 GB
DEBUG: Model output shape: torch.Size([16, 9])
DEBUG: Batch 6, RGB shape: torch.Size([16, 3, 256, 256]), HSI shape: torch.Size(

2025-04-30 10:43:26,712 - INFO - Epoch 1/10, Loss: 2.0870, Accuracy: 23.76%


DEBUG: Batch 1, RGB shape: torch.Size([16, 3, 256, 256]), HSI shape: torch.Size([16, 31, 256, 256])
DEBUG: GPU memory allocated: 0.19 GB
DEBUG: Model output shape: torch.Size([16, 9])
DEBUG: Batch 2, RGB shape: torch.Size([16, 3, 256, 256]), HSI shape: torch.Size([16, 31, 256, 256])
DEBUG: GPU memory allocated: 0.34 GB
DEBUG: Model output shape: torch.Size([16, 9])
DEBUG: Batch 3, RGB shape: torch.Size([16, 3, 256, 256]), HSI shape: torch.Size([16, 31, 256, 256])
DEBUG: GPU memory allocated: 0.33 GB
DEBUG: Model output shape: torch.Size([16, 9])
DEBUG: Batch 4, RGB shape: torch.Size([16, 3, 256, 256]), HSI shape: torch.Size([16, 31, 256, 256])
DEBUG: GPU memory allocated: 0.19 GB
DEBUG: Model output shape: torch.Size([16, 9])
DEBUG: Batch 5, RGB shape: torch.Size([16, 3, 256, 256]), HSI shape: torch.Size([16, 31, 256, 256])
DEBUG: GPU memory allocated: 0.33 GB
DEBUG: Model output shape: torch.Size([16, 9])
DEBUG: Batch 6, RGB shape: torch.Size([16, 3, 256, 256]), HSI shape: torch.Size(

...

2025-04-30 10:51:52,201 - INFO - Epoch 9/10, Loss: 1.5556, Accuracy: 43.37%


DEBUG: Batch 1, RGB shape: torch.Size([16, 3, 256, 256]), HSI shape: torch.Size([16, 31, 256, 256])
DEBUG: GPU memory allocated: 0.32 GB
DEBUG: Model output shape: torch.Size([16, 9])
DEBUG: Batch 2, RGB shape: torch.Size([16, 3, 256, 256]), HSI shape: torch.Size([16, 31, 256, 256])
DEBUG: GPU memory allocated: 0.34 GB
DEBUG: Model output shape: torch.Size([16, 9])
DEBUG: Batch 3, RGB shape: torch.Size([16, 3, 256, 256]), HSI shape: torch.Size([16, 31, 256, 256])
DEBUG: GPU memory allocated: 0.33 GB
DEBUG: Model output shape: torch.Size([16, 9])
DEBUG: Batch 4, RGB shape: torch.Size([16, 3, 256, 256]), HSI shape: torch.Size([16, 31, 256, 256])
DEBUG: GPU memory allocated: 0.34 GB
DEBUG: Model output shape: torch.Size([16, 9])
DEBUG: Batch 5, RGB shape: torch.Size([16, 3, 256, 256]), HSI shape: torch.Size([16, 31, 256, 256])
DEBUG: GPU memory allocated: 0.33 GB
DEBUG: Model output shape: torch.Size([16, 9])
DEBUG: Batch 6, RGB shape: torch.Size([16, 3, 256, 256]), HSI shape: torch.Size(

2025-04-30 10:52:54,642 - INFO - Epoch 10/10, Loss: 1.5489, Accuracy: 43.64%
2025-04-30 10:53:55,116 - INFO - Evaluation Accuracy: 45.56%


DEBUG: Evaluation Accuracy: 45.56%
DEBUG: Model saved to /content/drive/MyDrive/Colab Notebooks/PROJECT/models/mstpp_finetuned_new.pth


2025-04-30 10:53:55,174 - INFO - Model saved successfully
